In [20]:
import numpy as np
import os
import cv2
from scipy.spatial import distance
confThreshold = 0.75
nmsThreshold = 0.40
inpWidth = 576
inpHeight = 768

#Load names of classes and turn that into a list
classesFile = "coco.names"
classes = None

with open(classesFile,'rt') as f:
    classes = f.read().rstrip('\n').split('\n')

#Model configuration
modelConf = 'yolov3.cfg'
modelWeights = 'yolov3.weights'


def getOutputsNames(net):
    # Get the names of all the layers in the network
    layersNames = net.getLayerNames()
    # Get the names of the output layers, i.e. the layers with unconnected outputs
    return [layersNames[i[0] - 1] for i in net.getUnconnectedOutLayers()]

#Set up the net
net = cv2.dnn.readNetFromDarknet(modelConf, modelWeights)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)
fn = '000000' 
for count in range(1,len(os.listdir('sequence'))):
    a = ('sequence/'+fn[len(str(count)):]+str(count)+'.jpg')
    b = ('res_out_centroid/'+fn[len(str(count)):]+str(count)+'.jpg')
    frame = cv2.imread(a)#cv2.imread('images/000051.jpg')#cv2.VideoCapture(0)
    rects = []
    centroidlist = []
    blob = cv2.dnn.blobFromImage(frame, 1/255, (inpWidth, inpHeight), [0,0,0], 1, crop = False)
    net.setInput(blob)
    outs = net.forward (getOutputsNames(net))
    frameHeight = frame.shape[0]
    frameWidth = frame.shape[1]
    classIDs = []
    confidences = []
    boxes = []
    status = "People in Group"
    for out in outs:
        for detection in out:            
            scores = detection [5:]
            classID = np.argmax(scores)
            confidence = scores[classID]
            if confidence > confThreshold:
                centerX = int(detection[0] * frameWidth)
                centerY = int(detection[1] * frameHeight)
                width = int(detection[2]* frameWidth)
                height = int(detection[3]*frameHeight )
                left = int(centerX - width/2)
                top = int(centerY - height/2)
                classIDs.append(classID)
                confidences.append(float(confidence))
                boxes.append([left, top, width, height])
    indices = cv2.dnn.NMSBoxes (boxes,confidences, confThreshold, nmsThreshold )
    
    for i in indices:
        i = i[0]
        box = boxes[i]
        startX = box[0]
        startY = box[1]
        endX = box[2] + box[0]
        endY = box[3] + box[1]
        classId = classIDs[i]
        conf = confidences[i]
        label = '%.2f' % conf
        if classes:
            assert (classId < len(classes))
            label = '%s:%s' % (classes[classId], label)
        if label.startswith('person'):
            rects.append((startX, startY, endX, endY))
            centerX = startX + (endX - startX)/2
            centerY = endY + (startY - endY)/2
            centroidlist.append([int(centerX), int(centerY)])
    print(a)
    # loop over the tracked objects
    group = []
    dist_value = []
    # Loop to find the groups
    for e in range(len(centroidlist)):
        for f in range(len(centroidlist)):
            y = np.array(distance.cdist([centroidlist[e]],[centroidlist[f]]))
            if y < 55 and y != 0 and y not in dist_value:
                flag = 0
                if group != []:
                    for p in group:
                        z = [e,f]
                        if z[0] in p or z[1] in p:
                            p.extend(x for x in z if x not in p)
                            flag = 1
                if flag == 0:
                    group.append([e,f])
                dist_value.append(float(y))
    grp_count = 0
    if len(group) > 0:
        for i in group:
            grp_count +=len(i)
    Predestians_alone = len(centroidlist) - grp_count
    # Display on the frame with Predestians_alone and group with line and circle
    for i in centroidlist:
        cv2.circle(frame, (int(i[0]),int(i[1])), 3, (0, 0, 255), 3)
    for i in group:
        for j in i:
            for k in i:
                cv2.line(frame, (int(centroidlist[j][0]),int(centroidlist[j][1])),
                         (int(centroidlist[k][0]),int(centroidlist[k][1])), (0, 255, 0), 2)
                
    # construct a tuple of information we will be displaying on the
    info = [("Status", status),("Number of groups", len(group)),
            ("Pedestrians walk in groups", grp_count),("Pedestrians walk alone", Predestians_alone)]
    # Draw black background rectangle
    rect_x,rect_y,rect_w,rect_h = 0,0,250,80
    cv2.rectangle(frame, (rect_x, rect_x), (rect_x + rect_w, rect_y + rect_h), (0,0,0), -1)
    # loop over the info tuples and draw them on our frame
    for (i, (k, v)) in enumerate(info):
        text = "{}: {}".format(k, v)
        cv2.putText(frame, text, (10, ((i * 20) + 15)),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    cv2.imwrite(b, frame)
cv2.destroyAllWindows()

Using matplotlib backend: Qt5Agg
sequence/000001.jpg
sequence/000002.jpg
sequence/000003.jpg
sequence/000004.jpg
sequence/000005.jpg
sequence/000006.jpg
sequence/000007.jpg
sequence/000008.jpg
sequence/000009.jpg
sequence/000010.jpg
sequence/000011.jpg
sequence/000012.jpg
sequence/000013.jpg
sequence/000014.jpg
sequence/000015.jpg
sequence/000016.jpg
sequence/000017.jpg
sequence/000018.jpg
sequence/000019.jpg
sequence/000020.jpg
sequence/000021.jpg
sequence/000022.jpg
sequence/000023.jpg
sequence/000024.jpg
sequence/000025.jpg
sequence/000026.jpg
sequence/000027.jpg
sequence/000028.jpg
sequence/000029.jpg
sequence/000030.jpg
sequence/000031.jpg
sequence/000032.jpg
sequence/000033.jpg
sequence/000034.jpg
sequence/000035.jpg
sequence/000036.jpg
sequence/000037.jpg
sequence/000038.jpg
sequence/000039.jpg
sequence/000040.jpg
sequence/000041.jpg
sequence/000042.jpg
sequence/000043.jpg
sequence/000044.jpg
sequence/000045.jpg
sequence/000046.jpg
sequence/000047.jpg
sequence/000048.jpg
sequenc

sequence/000409.jpg
sequence/000410.jpg
sequence/000411.jpg
sequence/000412.jpg
sequence/000413.jpg
sequence/000414.jpg
sequence/000415.jpg
sequence/000416.jpg
sequence/000417.jpg
sequence/000418.jpg
sequence/000419.jpg
sequence/000420.jpg
sequence/000421.jpg
sequence/000422.jpg
sequence/000423.jpg
sequence/000424.jpg
sequence/000425.jpg
sequence/000426.jpg
sequence/000427.jpg
sequence/000428.jpg
sequence/000429.jpg
sequence/000430.jpg
sequence/000431.jpg
sequence/000432.jpg
sequence/000433.jpg
sequence/000434.jpg
sequence/000435.jpg
sequence/000436.jpg
sequence/000437.jpg
sequence/000438.jpg
sequence/000439.jpg
sequence/000440.jpg
sequence/000441.jpg
sequence/000442.jpg
sequence/000443.jpg
sequence/000444.jpg
sequence/000445.jpg
sequence/000446.jpg
sequence/000447.jpg
sequence/000448.jpg
sequence/000449.jpg
sequence/000450.jpg
sequence/000451.jpg
sequence/000452.jpg
sequence/000453.jpg
sequence/000454.jpg
sequence/000455.jpg
sequence/000456.jpg
sequence/000457.jpg
sequence/000458.jpg


In [23]:
import os
import cv2
img_array = []
fn = '000000' 
for count in range(1,len(os.listdir('res_out_centroid'))):
    a = ('res_out_centroid/'+fn[len(str(count)):]+str(count)+'.jpg')
    img = cv2.imread(a)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)

out = cv2.VideoWriter('Task_3_centroid.avi',cv2.VideoWriter_fourcc(*'DIVX'), 5, size)

for i in range(len(img_array)):
    out.write(img_array[i])
out.release()